In [0]:
"""

  Text Analytic (Emotion) with TensorFlow
 
  Copyright 2020  I Made Agus Dwi Suarjaya
                  Ni Luh Putu Diah Putri M
                  Gede Ocha Dipa Ananda

  Description     : Try to analyze Tweets with TensorFlow and classify into 5 emotions (anger, happiness, sadness, love, fear)
  Dataset source  : https://raw.githubusercontent.com/meisaputri21/Indonesian-Twitter-Emotion-Dataset/master/Twitter_Emotion_Dataset.csv

"""

#Setup
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

import csv
import time
import ast

import numpy as np
import pandas as pd

#Enable eager for easy experiment
tf.compat.v1.enable_eager_execution()

In [0]:
#--------------------------------------------------------------------------------------------------------------------------

#Get dataset by url into panda dataframe
dataset_url = 'https://raw.githubusercontent.com/meisaputri21/Indonesian-Twitter-Emotion-Dataset/master/Twitter_Emotion_Dataset.csv'
df1 = pd.read_csv(dataset_url)
#print (df1)                                                #Take a look at the dataframe

In [0]:
#Make category dictionary for the label
c = df1.label.astype('category')
d = dict(enumerate(c.cat.categories))
#print (d)                                                  #Take a look at the dictionary                                           

In [0]:
#Convert categories label to codes
df1['label'] = pd.Categorical(df1['label'])
df1['label'] = df1.label.cat.codes

#df1.head()                                                 #Take a look at the head of dataframe

In [0]:
#Split then combine dataframe into tensorflow dataset
target = df1.label
featu = df1.tweet

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((featu.values, target.values))

In [0]:
#for feat, targ in dataset.take(5):
#  print ('Features: {}, Target: {}'.format(feat, targ))    #Take a look at the dataset

In [0]:
#Tokenize the strings into tokens then make vocabulary set of the dataset
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in dataset:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
#vocab_size                                                 #Take a look at the vocabulary size
#vocabulary_set                                             #Take a look at the content of vocabulary set

In [0]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [0]:
#For testing the encoder with example text
#example_text = next(iter(dataset))[0].numpy()
#print(example_text)
#encoded_example = encoder.encode(example_text)
#print(encoded_example)

In [0]:
#Function to encode text into tokenized text
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

In [0]:
#Function to encode and map dataset into encoded dataset
def encode_map_fn(text, label):
  encoded_text, label = tf.py_function(encode, inp=[text, tf.cast(label, tf.int64)], Tout=(tf.int64, tf.int64))

  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


encoded_dataset = dataset.map(encode_map_fn)

In [0]:
#for ex in encoded_dataset.take(5):
#  print(ex)                                              #Take a look at the encoded dataset

In [0]:
#Shuffle the dataset
encoded_dataset = encoded_dataset.shuffle(50000, reshuffle_each_iteration=False)

In [0]:
#Split dataset into train dan test dataset
train_dataset = encoded_dataset.take(4000)
test_dataset = encoded_dataset.skip(4000)

In [0]:
#Padd the dataset to have the same length
train_dataset = train_dataset.padded_batch(40, 
                                           padded_shapes=([None],[]))
test_dataset = test_dataset.padded_batch(40, 
                                         padded_shapes=([None],[]))

In [0]:
#Take a look at the padded dataset
#sample_text, sample_labels = next(iter(test_dataset))
#
#for i in range(10):
#  print(sample_text[i], sample_labels[i])

In [0]:
#Make the model and set the hyperparameters
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size + 1, 64))            #Add 1 to vocabulary size because of padding
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

for units in [64,16]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

model.add(tf.keras.layers.Dense(5))                                 #Output layer. The first argument is the number of labels.

model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

In [22]:
#Set the hyperparameters and train the model
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
100/100 [==============================] - 14s 140ms/step - loss: 1.5344 - acc: 0.2770 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/10
100/100 [==============================] - 11s 106ms/step - loss: 1.0991 - acc: 0.5297 - val_loss: 1.3864 - val_acc: 0.3840
Epoch 3/10
100/100 [==============================] - 11s 106ms/step - loss: 0.7881 - acc: 0.6842 - val_loss: 1.6675 - val_acc: 0.3840
Epoch 4/10
100/100 [==============================] - 11s 106ms/step - loss: 0.7010 - acc: 0.7055 - val_loss: 1.5811 - val_acc: 0.4564
Epoch 5/10
100/100 [==============================] - 11s 107ms/step - loss: 0.4939 - acc: 0.8207 - val_loss: 1.9851 - val_acc: 0.4364
Epoch 6/10
100/100 [==============================] - 11s 110ms/step - loss: 0.3179 - acc: 0.8953 - val_loss: 2.1108 - val_acc: 0.4738
Epoch 7/10
100/100 [==============================] - 11s 112ms/step - loss: 0.2938 - acc: 0.9022 - val_loss: 1.9635 - val_acc: 0.4913
Epoch 8/10
100/100 [===========================

In [0]:
#Evaluate the model
eval_loss, eval_acc = model.evaluate(test_dataset)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

In [0]:
#Save the model (Optional for Transfer Learning)
t = time.time()

export_path_model = "./{}_model".format(int(t))
model.save(export_path_model, save_format='h5')

In [0]:
#Save the encoder (Optional for Transfer Learning)
export_path_encoder = "./{}_encoder".format(int(t))
encoder.save_to_file(export_path_encoder)

In [0]:
#Save the dictionary (Optional for Transfer Learning)
export_path_dict = "./{}_dict".format(int(t))
with open(export_path_dict, 'w') as outfile:
    outfile.write(str(d))

In [0]:
#--------------------------------------------------------------------------------------------------------------------------

#Load the model (Optional for Transfer Learning)
reloaded_model = tf.keras.models.load_model(export_path_model)
model = reloaded_model

In [0]:
#Load the encoder (Optional for Transfer Learning)
encoder = tfds.features.text.TokenTextEncoder.load_from_file(export_path_encoder)

In [0]:
#Load the dictionary (Optional for Transfer Learning)
with open(export_path_dict) as dict_file:
    d = ast.literal_eval(dict_file.readline())

In [91]:
#Classify some tweets with model predict
tweet = []
tweet.append('Tahukah kamu, bahwa saat itu papa memejamkan matanya dan menahan gejolak dan batinnya. Bahwa papa sangat ingin mengikuti keinginanmu tapu lagi-lagi dia HARUS menjagamu?')
tweet.append('[Idm] My, masa gua tadi ketemu tmn SD yg pas SD ngejar gua dan ngasih surat tiap minggunya, asdfghjkl bgt, gk tau knp ngerasa takut gua :v hadeuh jaman SD ngerti apa coba :v')
tweet.append('Sedih bny penulisan resep yg tidak baku sdm, sdt, ruas, sejumput, secukupnya, even biji/buah termasuk tidak baku :(')
tweet.append('Paling nyampah org suka compare kan aku dgn org lain, dia dia ah aku aku ah. Tak suka boleh blah lah')
tweet.append('Agak telat ramai nya ya dok...sudah paham sejak lama banget jadi geli aja baru pada ramai sekarang hehehe...')

for text in range(len(tweet)):
  predictions = model.predict(encoder.encode(tweet[text]))
  predictions[0]
  print(d[np.argmax(predictions[0])], ' <- ', tweet[text])

love  <-  Tahukah kamu, bahwa saat itu papa memejamkan matanya dan menahan gejolak dan batinnya. Bahwa papa sangat ingin mengikuti keinginanmu tapu lagi-lagi dia HARUS menjagamu?
fear  <-  [Idm] My, masa gua tadi ketemu tmn SD yg pas SD ngejar gua dan ngasih surat tiap minggunya, asdfghjkl bgt, gk tau knp ngerasa takut gua :v hadeuh jaman SD ngerti apa coba :v
sadness  <-  Sedih bny penulisan resep yg tidak baku sdm, sdt, ruas, sejumput, secukupnya, even biji/buah termasuk tidak baku :(
anger  <-  Paling nyampah org suka compare kan aku dgn org lain, dia dia ah aku aku ah. Tak suka boleh blah lah
happy  <-  Agak telat ramai nya ya dok...sudah paham sejak lama banget jadi geli aja baru pada ramai sekarang hehehe...
